In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import os
import cv2

In [14]:
train_dir = '/Users/anonymous/Desktop/ml/data/chest_xray/chest_xray/train'
val_dir = '/Users/anonymous/Desktop/ml/data/chest_xray/chest_xray/val'
test_dir = '/Users/anonymous/Desktop/ml/data/chest_xray/chest_xray/test'

In [15]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150
def get_training_data(data_dir):
    images = [] 
    labels_list = []
    
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                flattened_arr = resized_arr.flatten()  # Flatten the image array
                images.append(flattened_arr)
                labels_list.append(class_num)
            except Exception as e:
                print(e)
    
    images_array = np.array(images)
    labels_array = np.array(labels_list)
    
    data = np.column_stack((images_array, labels_array))
    
    return data

In [16]:
train = get_training_data(train_dir)
val = get_training_data(val_dir)
test = get_training_data(test_dir)

OpenCV(4.9.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.9.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.9.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

OpenCV(4.9.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



In [6]:
def oversample_minority(data):
    # Count the number of normal and disease images
    normal_indices = np.where(data[:, -1] == 1)[0]  # Indices of normal images
    disease_indices = np.where(data[:, -1] == 0)[0]  # Indices of disease images

    normal_count = len(normal_indices)
    disease_count = len(disease_indices)

    # Calculate the number of samples needed to balance the classes
    oversample_count = disease_count - normal_count

    if oversample_count <= 0:
        print("No oversampling needed.")
        return data

    # Randomly sample some normal images to add to the dataset
    oversampled_indices = np.random.choice(normal_indices, size=oversample_count, replace=True)

    # Concatenate the oversampled indices with the original dataset
    oversampled_data = data[oversampled_indices]
    balanced_data = np.vstack((data, oversampled_data))

    return balanced_data

In [17]:
train = oversample_minority(train)

In [8]:
def split_data(data):
    x_train = []  # Feature array (pixel data)
    y_train = []  # Label array

    for entry in data:
        # Append image data to x_train
        x_train.append(entry[:-1])  # Exclude the last element (label) from the entry
        
        # Append label to y_train
        y_train.append(entry[-1])  # Last element in the entry is the label

    # Convert lists to NumPy arrays for efficiency
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    return x_train, y_train

In [18]:
x_train, y_train = split_data(train)
x_val, y_val = split_data(val)
x_test, y_test = split_data(test)

In [19]:
# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

In [20]:
x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [21]:
# Training the Model
model = Sequential()
model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 1 , activation = 'sigmoid'))
model.compile(optimizer = "rmsprop" , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 150, 150, 32)      320       
                                                                 
 batch_normalization_5 (Bat  (None, 150, 150, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 75, 75, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 dropout_4 (Dropout)         (None, 75, 75, 64)        0         
                                                                 
 batch_normalization_6 (Bat  (None, 75, 75, 64)       

In [22]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)

In [23]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
history = model.fit(datagen.flow(x_train,y_train, batch_size = 32) ,epochs = 12 , validation_data = datagen.flow(x_val, y_val) ,callbacks = [learning_rate_reduction])

Epoch 1/12
243/243 [==============================] - 153s 624ms/step - loss: 0.4558 - accuracy: 0.8539 - val_loss: 14.4205 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 2/12
243/243 [==============================] - 214s 879ms/step - loss: 0.2545 - accuracy: 0.9093 - val_loss: 3.0093 - val_accuracy: 0.5625 - lr: 0.0010
Epoch 3/12
243/243 [==============================] - 157s 647ms/step - loss: 0.1919 - accuracy: 0.9317 - val_loss: 1.5276 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/12
243/243 [==============================] - ETA: 0s - loss: 0.1830 - accuracy: 0.9375
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
243/243 [==============================] - 193s 793ms/step - loss: 0.1830 - accuracy: 0.9375 - val_loss: 2.2844 - val_accuracy: 0.5625 - lr: 0.0010
Epoch 5/12
243/243 [==============================] - 161s 664ms/step - loss: 0.1285 - accuracy: 0.9550 - val_loss: 1.1048 - val_accuracy: 0.7500 - lr: 3.0000e-04
Epoch 6/12
225/243 [===================